# Project 03: Gibbs Sampling (Random Algorithm)

In [3]:
import random
import numpy as np
import bamnostic as bs
import seqlogo
from pprint import pprint

#import function for building sequence motif & idenfitying seqs matching to motif
from data_readers import *
import seq_ops
import motif_ops
import numba

from joblib import Parallel, delayed

from utils import utils
from sequence_database import sequence_box
from collections import Counter, defaultdict

from scipy.special import softmax 

print("Great, everything is up to date")

Great, everything is up to date


In [4]:
bases = ["A", "T", "C", "G", "N"]
encode_map = utils.init_base_encoding_map()
import time

---
## Implement Gibbs Sampler


Gibbs sampling is a MCMC approach to identify enrichments. Here we will implement a method to identify motifs from a set of regions. 

Important considerations:
- We will need to score each sequence with a PWM using the `score_kmer()` or `score_sequence()` functions
    - You will need to investigate into the help documenation and libraries to identify how best to use these functions. 
- These sites are often not strand-specific and so both scores on the negative as well as positive strand should be considered
- To select a random sequence, use `random.randint()` or `numpy.random.randint()`
- To select a new position $m$ (as defined below) use `random.choices()` or `numpy.random.choice()`  

Assumptions: 
- We know $k$ as the length of expected motif
- Each sequence contains the motif



```
GibbsMotifFinder(DNA, k-length)
    random pick of k-length sequences from each line of DNA as Motifs
    for j ← 1 to 10000 or Motifs stops changing
        i ← Random(N) where N is number of DNA entries
        PWM ← PWM constructed from all Motifs except for Motifi
        Motifi ← select position m from PWM-scored k-mers in DNAi in probabilistic fashion from score distribution
    return PFM
```

Probability of chosing position $m = \frac{A_{m}}{\sum_{l}A_{l}}$ for positions $l$ in DNAi


**Note:** I have also added a function to `motif_ops.py` that will calculate the information content of your motifs. This is useful to observe the progression of your Gibbs sampler as well as a measure of convergence. You can use this function as `IC = pfm_ic(pfm)`. You should expect a slow increase of IC until it plateaus such as in the plot below from your lecture slides:

<center><img src='figures/Gibbs_Sampling.png'/ width=600px></center>

---
# Setting up the data

This BAM file contains a subsampled set of aligned ChIP‑seq reads from a p53 immunoprecipitation experiment in human K562 cells treated with the anthracycline drug daunorubicin. The original SRA experiment SRX5865974 (run SRR9090854) reports 31.2 million Illumina NextSeq 500 ChIP‑seq read pairs from K562 wild‑type cells exposed to daunorubicin, using a p53 antibody to pull down p53‑bound chromatin fragments before library preparation and sequencing. [pmc.ncbi.nlm.nih](https://pmc.ncbi.nlm.nih.gov/articles/PMC4366240/)

### What the data represent

- Biological system: human K562 leukemia cells (Homo sapiens) treated with daunorubicin, a DNA‑damaging chemotherapeutic known to stabilize and activate p53. [pmc.ncbi.nlm.nih](https://pmc.ncbi.nlm.nih.gov/articles/PMC6561911/)
- Assay: ChIP‑seq using an antibody against p53, so reads should be enriched around genomic regions where p53 is bound after drug treatment. [pmc.ncbi.nlm.nih](https://pmc.ncbi.nlm.nih.gov/articles/PMC4526040/)
- Sequencing: Illumina NextSeq 500, with the raw run SRR9090854 corresponding to experiment SRX5865974. [github](https://github.com/ncbi/sra-tools/issues/213)
- BAM file: `SRR9090854.subsampled_5pct.bam` is a coordinate‑sorted alignment file containing ~5% of the original mapped reads, typically created by random down‑sampling the full BAM to reduce file size and speed up exploratory analyses while preserving the overall distribution of p53 binding events. [ecseq](https://www.ecseq.com/support/ngs-snippets/how-to-extract-a-list-of-specific-read-IDs-from-a-BAM-file)

### How `bamnostic` is used and what your code does

The `bamnostic` package provides a pure‑Python interface to BAM files that mirrors the `pysam` API, including an `AlignmentFile` class whose iterator yields `AlignedSegment` objects representing individual aligned reads. In your code: [bamnostic.readthedocs](https://bamnostic.readthedocs.io/en/latest/bamnostic.html)

```python
bam_path = "data/SRR9090854.subsampled_5pct.bam"

seqs = [read.seq for read in bs.AlignmentFile(bam_path)]
```

- `bs.AlignmentFile(bam_path)` opens the BAM file as an `AlignmentFile` object in binary read mode (default `'rb'`), reading the BAM header (reference contigs, read groups, etc.) and preparing a streaming interface to all aligned records. [github](https://github.com/betteridiot/bamnostic/blob/master/docs/source/quickstart.rst)
- Iterating over `AlignmentFile` (`for read in bs.AlignmentFile(...)`) returns each aligned read as a `bamnostic.AlignedSegment` object, which exposes properties analogous to SAM fields such as query name, flags, reference name, position, mapping quality, CIGAR string, and the original sequencing **sequence**. [bamnostic.readthedocs](https://bamnostic.readthedocs.io/en/latest/bamnostic.html)
- The `read.seq` attribute is the query (read) sequence string stored in the BAM, corresponding to the full read sequence (including any unaligned bases), as opposed to `query_alignment_sequence`, which would only contain the aligned portion. [bamnostic.readthedocs](https://bamnostic.readthedocs.io/en/latest/bamnostic.html)
- The list comprehension `[read.seq for read in ...]` consumes the entire BAM stream and collects the nucleotide sequences from every subsampled ChIP‑seq read into a Python list `seqs`, which can then be used for downstream tasks such as motif discovery, k‑mer analysis, or quality checks on read content. [ucdavis-bioinformatics-training.github](https://ucdavis-bioinformatics-training.github.io/2022-Feb-Introduction-To-Python-For-Bioinformatics/python/python5)

In summary, your dataset is a 5% random sample of p53‑ChIP‑seq alignments from daunorubicin‑treated K562 cells, and the `bamnostic` code opens the subsampled BAM and extracts the raw read sequences from each aligned fragment into memory as a list.

---
# Project Start

# Functions

In [11]:
def GibbsMotifFinder(seqs=None, k=6, n_rows=3083497, mode="norm", speed="pythonic", p_method="softmax", rtol=1e-5, atol=1e-10, max_iter=1024, seed=None, toprint=True):
    '''
    Function to find a pfm from a list of strings using a Gibbs sampler
    
    Args: 
        seqs (str list): a list of sequences, not necessarily in same lengths
        k (int): the length of motif to find
        seed (int, default=None): seed for np.random

    Returns:
        pfm (numpy array): dimensions are 4xlength
    '''

    '''
    -----------------------------------------------INITIALIZATION---------------------------------------
    '''

    # Use rng to make random samples/selections/numbers
    # Example: randint = rng.integer(1, 10)
    random.seed(seed)
    rng = np.random.default_rng(seed)

    seqs, indptr = utils.io_monster(mode)
    motifs, midx = utils.fast_init(seqs, n_rows, indptr, k)
    seq_box = sequence_box(indptr, seqs, midx, motifs, k)

    # bg = seq_box.get_bg()


    '''
    -----------------------------------------------ITERATION LOOP---------------------------------------
    '''
    i = 0
    if speed == "pythonic:":

        #SLICING SUBSET
        mots = seq_box.motifs[:100]
        sample_list = []
        for j in range(len(mots)):
            sample_list.append(utils.decode_sequence(mots[j]))

        #ALTERNATE METHOD
        mots = seq_box.get_str_list_format_motifs()

        converged = False
        #i = 0
        while converged == False and i < max_iter:

            mot_pick = np.random.randint(0, len(sample_list))
            sample_list.pop(mot_pick)
            pfm = motif_ops.build_pfm(sample_list, k)
            pwm = motif_ops.build_pwm(pfm)

            seq = utils.decode_sequence(seq_box[mot_pick])
            rev_seq = seq_ops.reverse_complement(seq)

            scores = []
            for x in range(len(seq) - k):
                score = motif_ops.score_kmer(seq[x:x+k], pwm)
                scores.append(score)
            
            for x in range(len(seq) - k):
                score = motif_ops.score_kmer(rev_seq[x:x+k], pwm)
                scores.append(score)
            
            prob = softmax(scores)
            #print(prob)
            idx = np.random.choice(np.arange(len(scores)), p=prob)

            quotient, remainder = divmod(idx, len(seq) - k - 1)
            if quotient == 0:
                new_motif = seq[remainder:remainder+k]
            else:
                new_motif = rev_seq[remainder:remainder+k]

            seq_box.midx[mot_pick] = remainder
            sample_list.insert(mot_pick, new_motif)

            if i > 0:
                if np.allclose(pwm, pwm_old, ):
                    converged = True
                    pprint(pwm-pwm_old)

            pwm_old = pwm.copy()
            i += 1

        
            
    elif speed == "fast":
        if toprint:
            pfm = seq_box.get_pfm()
            pprint(pfm)
        if toprint:
            print(f"beginning fast iteration")
        converged = False
        #i = 0

        while converged == False and i < max_iter:
            print(f"\rIteration {i}", end="", flush=True)
            fwd_seq = seq_box.select_random_motif()
            pfm = seq_box.get_pfm(to_mask=True)
            pwm = motif_ops.build_pwm(pfm)
            rev_seq = utils.fast_complement(fwd_seq)
            fwd, rev = utils.fast_subdivide(fwd_seq, rev_seq, k)
            best_motif = utils.choose_best(fwd, rev, pwm, p_method)
            seq_box.update_motifs(best_motif)

            if i > 0:
                if np.allclose(pwm, pwm_old, rtol, atol):
                    converged = True
            pwm_old = pwm.copy()
            
            i += 1

    output = seq_box.get_str_list_format_motifs()

    if toprint:
        pfm = seq_box.get_pfm()
        print()
        pprint(pfm)
        if i == max_iter:
                print("\nalgorithm did not converge :(((")

        print(f'\nAfter {i} iterations, final motif list: {output[:20]}')

    return pfm
    #return output

In [6]:
def consensus(results, top_k=20):
    votes = Counter()
    for result in results:
        votes.update(Counter(result))
    return votes.most_common(top_k)

---
# Driver Program
Don't change any of the code here. If you have completed the project by following the coding by contract, the following code should work.

In [7]:
#create the seqs array, just need to do 1 time and it will take a while. For Vic it took ~3-5 minutes.
bam_path = "data/SRR9090854.subsampled_5pct.bam"

seqs = [read.seq for read in bs.AlignmentFile(bam_path)]


In [9]:
#Check that seqs has initialized (and view some data)
pprint(seqs[0:10])

['CCTAACCCTAACCCTAACCCTAACCCTATCCAGATCG',
 'ACCCTAACCCTAACCCAAACCCTAACCCTAACAGATC',
 'CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC',
 'CTAACCCTAACCCTAACCCTAACCCTAACCCTAAC',
 'CTATCCCTAACCCTAACCCTAACCCTAACCCTAACC',
 'CGATATCCTAACCCTAACCCTAACCCTAACCCTAACC',
 'ATCTACCCTAACCCTAACCCTAACCCTAACCCTAAC',
 'TACCCCTAACCCTAACCCTAACCCTAACCCTAACCCT',
 'CCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCC',
 'CCTAACCCTAACCCTCGCGGTACCCTCAGCCGGCCCG']


In [13]:
#runs a lot of times to check convergence - you will get a "consensus" sequence by running this

threads = 16

results = Parallel(n_jobs=threads)([delayed(GibbsMotifFinder)(k=6, speed="fast", rtol=1e-5, max_iter=6000) for _ in range(threads)])

top_k = consensus(results)
print(top_k)

[('AAAAAA', 118190), ('TTTTTT', 117764), ('ATTTTT', 64784), ('AAAAAT', 64463), ('TATTTT', 53983), ('AAAATA', 53425), ('TTTTTA', 50967), ('TTTCTT', 50642), ('TAAAAA', 50264), ('AGAAAA', 49118), ('TTTTCT', 48549), ('TTCTTT', 48204), ('AGGCTG', 47928), ('CAGCCT', 47603), ('AAGAAA', 47555), ('TTTTTG', 46688), ('AAAGAA', 46036), ('CAAAAA', 45589), ('CCTCCC', 45276), ('TTATTT', 45180)]


In [12]:
# driver code - generate your pwm to get seqlogo
pwm = GibbsMotifFinder()

processed_data.npz exists, reading from path
data ready :)

array([[851013, 850328, 853049, 853306, 853173, 848130],
       [694103, 691768, 688776, 685085, 684092, 685300],
       [685299, 685037, 685413, 687016, 690784, 696087],
       [853082, 856364, 856259, 858090, 855448, 853980]], dtype=int32)

After 0 iterations, final motif list: ['ACCCTA', 'CTAACC', 'CTAACC', 'CCCTAA', 'ACCCTA', 'TCCTAA', 'CCTAAC', 'CCTAAC', 'AACCCT', 'TCGCGG', 'CCCGGG', 'ACGCAG', 'GCCTAC', 'CTCAGT', 'AGGGCA', 'TGCAGG', 'CTCTCT', 'TGGCCA', 'GCAGGG', 'TATAGT']


In [13]:
pprint(pwm)

None


In [23]:
cpm =seqlogo.seqlogo(seqlogo.CompletePm(pfm = results))

TypeError: pm_filename_or_array` must be a filename, `np.ndarray`, `pd.DataFrame`, or `Pm`

In [22]:
print(cpm.pfm)

NameError: name 'cpm' is not defined

In [14]:
#Marcus' code:

# Run the gibbs sampler:
promoter_pfm = GibbsMotifFinder(seqs,10 )

# Plot the final pfm that is generated: 
seqlogo.seqlogo(seqlogo.CompletePm(pfm = promoter_pfm.T))

processed_data.npz exists, reading from path
data ready :)

array([[851003, 851150, 852573, 853620, 853020, 853214, 853420, 852598,
        852583, 848144],
       [696653, 693485, 693093, 689424, 687978, 685962, 682194, 682602,
        682572, 684368],
       [684557, 683401, 682597, 682447, 685769, 687684, 690444, 692167,
        694021, 697762],
       [851284, 855461, 855234, 858006, 856730, 856637, 857439, 856130,
        854321, 853223]], dtype=int32)

After 0 iterations, final motif list: ['CCTAACCCTA', 'AAACCCTAAC', 'CCCTAACCCT', 'ACCCTAACCC', 'ACCCTAACCC', 'CGATATCCTA', 'ACCCTAACCC', 'CCCTAACCCT', 'CCCTCGCGGT', 'AACCCTCGCG', 'CGGGTCTGAC', 'AATCTGTGCA', 'CGCAGAGACG', 'AATCAGAAAA', 'GGGCTCTCTT', 'GGCGACTAGG', 'AACTGCAGGG', 'TCTCTTGCTT', 'GGGCACTGCA', 'GCCCTCTTGC']


AttributeError: 'NoneType' object has no attribute 'T'